# 問題39: Zipfの法則

単語の出現頻度順位を横軸，その出現頻度を縦軸として，両対数グラフをプロットせよ．

In [ ]:
# 問題30で実装した関数を読み込む
import sys
sys.path.append('../..')  # 親ディレクトリをパスに追加
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 日本語表示のための設定
plt.rcParams['font.family'] = 'IPAexGothic'

# 形態素解析結果を読み込む関数
def load_mecab_result(file_path):
    """
    MeCabの解析結果ファイルを読み込み、各形態素を辞書のリストとして返す関数
    
    Args:
        file_path (str): MeCab出力ファイルのパス
        
    Returns:
        list: 文のリスト。各文は形態素（辞書）のリスト
    """
    sentences = []
    current_sentence = []
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # EOSは文の区切り
            if line == 'EOS\n':
                if current_sentence:
                    sentences.append(current_sentence)
                    current_sentence = []
                continue
                
            # 空行をスキップ
            if line == '\n':
                continue
                
            # タブで分割して表層形とそれ以外の情報に分ける
            try:
                surface, info = line.split('\t')
                
                # カンマで分割して品詞情報などを取得
                info_items = info.split(',')
                
                # 形態素情報を辞書として格納
                morpheme = {
                    'surface': surface,
                    'base': info_items[6],
                    'pos': info_items[0],
                    'pos1': info_items[1]
                }
                
                current_sentence.append(morpheme)
            except:
                # 不正な形式の行をスキップ
                continue
    
    # 最後の文が追加されていない場合に追加
    if current_sentence:
        sentences.append(current_sentence)
    
    return sentences

# 単語の出現頻度を計算する関数
def count_word_frequency(sentences):
    """
    形態素解析結果から単語の出現頻度を計算する関数
    
    Args:
        sentences (list): 文のリスト。各文は形態素（辞書）のリスト
        
    Returns:
        Counter: 単語の出現頻度を格納したCounterオブジェクト
    """
    # 全ての形態素の表層形を抽出
    words = [morpheme['surface'] for sentence in sentences for morpheme in sentence]
    
    # 単語の出現回数をカウント
    word_counts = Counter(words)
    
    return word_counts

# Zipfの法則をプロットする関数
def plot_zipf_law(word_counts, top_n=None):
    """
    単語の出現頻度順位と出現頻度の関係を両対数グラフでプロットする関数
    
    Args:
        word_counts (Counter): 単語の出現頻度を格納したCounterオブジェクト
        top_n (int): プロットする単語の数（上位n語）。Noneの場合は全単語
    """
    # 出現頻度の降順でソート
    sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
    
    # 上位n語に制限（指定がある場合）
    if top_n is not None:
        sorted_word_counts = sorted_word_counts[:top_n]
    
    # 順位と頻度のリストを作成
    ranks = list(range(1, len(sorted_word_counts) + 1))
    frequencies = [count for word, count in sorted_word_counts]
    
    # データフレームに変換
    df = pd.DataFrame({'順位': ranks, '出現頻度': frequencies})
    
    # 理論値の計算（Zipfの法則: f ∝ 1/r）
    # 最も頻度の高い単語の頻度を基準に理論値を計算
    max_freq = frequencies[0]
    theoretical = [max_freq / r for r in ranks]
    df['理論値'] = theoretical
    
    # 両対数グラフのプロット
    plt.figure(figsize=(12, 8), dpi=300)
    
    # 実測値のプロット
    plt.loglog(df['順位'], df['出現頻度'], 'o', markersize=3, alpha=0.5, label='実測値')
    
    # 理論値のプロット
    plt.loglog(df['順位'], df['理論値'], 'r-', alpha=0.7, label='理論値 (Zipfの法則)')
    
    # グラフの装飾
    plt.title('Zipfの法則の検証', fontsize=16)
    plt.xlabel('出現頻度順位', fontsize=14)
    plt.ylabel('出現頻度', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True, which="both", ls="-", alpha=0.2)
    
    plt.tight_layout()
    
    return plt.gcf()

In [ ]:
# 形態素解析結果の読み込み
file_path = '../../data/neko.txt.mecab'
sentences = load_mecab_result(file_path)

# 単語の出現頻度を計算
word_counts = count_word_frequency(sentences)

# Zipfの法則をプロット（全単語）
fig = plot_zipf_law(word_counts)
plt.show()

# Zipfの法則をプロット（上位1000語）
fig2 = plot_zipf_law(word_counts, top_n=1000)
plt.title('Zipfの法則の検証（上位1000語）', fontsize=16)
plt.show()

# 上位100語の順位と頻度を表形式で表示
sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)
ranks = list(range(1, 101))
words = [word for word, count in sorted_word_counts[:100]]
frequencies = [count for word, count in sorted_word_counts[:100]]

df = pd.DataFrame({'順位': ranks, '単語': words, '出現頻度': frequencies})
display(df.head(20))

# 理論値との比較
max_freq = frequencies[0]
theoretical = [max_freq / r for r in ranks]
df['理論値'] = theoretical
df['実測値/理論値'] = df['出現頻度'] / df['理論値']
display(df[['順位', '単語', '出現頻度', '理論値', '実測値/理論値']].head(20))

## 解説

この問題では、単語の出現頻度順位と出現頻度の関係を両対数グラフでプロットし、Zipfの法則を検証しました。

### Zipfの法則とは

Zipfの法則（ジップの法則）は、言語学者のジョージ・キングズリー・ジップによって提唱された経験則で、自然言語における単語の出現頻度に関する法則です。この法則によれば、単語の出現頻度はその出現頻度順位に反比例します。

数式で表すと：
$$f \propto \frac{1}{r}$$

ここで、$f$は単語の出現頻度、$r$はその単語の出現頻度順位です。つまり、最も頻度の高い単語の出現頻度を$f_1$とすると、2番目に頻度の高い単語の出現頻度は約$f_1/2$、3番目は約$f_1/3$、...となります。

### 実装のポイント

1. **両対数グラフ**: Zipfの法則は両対数グラフ上で直線になります。これは、$\log(f) = -\log(r) + C$（Cは定数）という関係を示しています。

2. **理論値の計算**: 最も頻度の高い単語の頻度を基準に、Zipfの法則に基づく理論値を計算しています。

3. **実測値との比較**: 実測値と理論値を同じグラフ上にプロットして、Zipfの法則がどの程度成り立っているかを視覚的に確認しています。

### 結果の考察

グラフから、以下のような特徴が観察できます：

1. **全体的な傾向**: 両対数グラフ上で、単語の出現頻度と順位の関係はおおむね直線に近い形になっています。これは、Zipfの法則が概ね成り立っていることを示しています。

2. **高頻度語**: 最も頻度の高い単語（主に助詞や助動詞）は、理論値よりもやや頻度が高い傾向があります。これは、日本語の文法構造上、これらの機能語が非常に頻繁に使用されるためと考えられます。

3. **中頻度語**: 中程度の頻度の単語は、理論値に比較的近い値を示しています。

4. **低頻度語**: 非常に頻度の低い単語（ハプックスレゴメナなど）は、理論値からやや外れる傾向があります。これは、サンプルサイズの制約や、専門用語・固有名詞の存在などが影響していると考えられます。

### Zipfの法則の意義

Zipfの法則は、自然言語だけでなく、都市の人口分布、企業の規模分布、ウェブサイトのアクセス数など、様々な現象で観察されます。この普遍性は、複雑系における自己組織化や最小労力の原理などと関連していると考えられています。

自然言語処理においては、Zipfの法則は語彙の分布特性を理解し、モデル化する上で重要です。例えば、低頻度語の扱い（スムージングや未知語処理）や、語彙サイズの推定などに応用されています。